In [1]:
import pytz
import os


DEFAULT_TZ = pytz.FixedOffset(540)  # GMT+09:00; Asia/Seoul

PATH_DATA = 'data/D'
PATH_ESM = os.path.join(PATH_DATA, 'EsmResponse.csv')
PATH_PARTICIPANT = os.path.join(PATH_DATA, 'UserInfo.csv')
PATH_SENSOR = os.path.join(PATH_DATA, 'Sensor')

PATH_INTERMEDIATE = os.path.join('/home/uzair/Stress/StressDetection_Collaboration/Data_Processing_D1/data/intermediate')
RANDOM_STATE = 42

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import cloudpickle
import ray
from datetime import datetime
from contextlib import contextmanager
import warnings
import time


def load(path: str):
    with open(path, mode='rb') as f:
        return cloudpickle.load(f)

    
def dump(obj, path: str):
    with open(path, mode='wb') as f:
        cloudpickle.dump(obj, f)
        
    
def log(msg: any):
    print('[{}] {}'.format(datetime.now().strftime('%y-%m-%d %H:%M:%S'), msg))


def summary(x):
    x = np.asarray(x)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        n = len(x)
        # Here, uppercase np.dtype.kind corresponds to non-numeric data.
        # Also, we view the boolean data as dichotomous categorical data.
        if x.dtype.kind.isupper() or x.dtype.kind == 'b': 
            cnt = pd.Series(x).value_counts(dropna=False)
            card = len(cnt)
            cnt = cnt[:20]                
            cnt_str = ', '.join([f'{u}:{c}' for u, c in zip(cnt.index, cnt)])
            if card > 30:
                cnt_str = f'{cnt_str}, ...'
            return {
                'n': n,
                'cardinality': card,
                'value_count': cnt_str
            }
        else: 
            x_nan = x[np.isnan(x)]
            x_norm = x[~np.isnan(x)]
            
            tot = np.sum(x_norm)
            m = np.mean(x_norm)
            me = np.median(x_norm)
            s = np.std(x_norm, ddof=1)
            l, u = np.min(x_norm), np.max(x)
            conf_l, conf_u = st.t.interval(0.95, len(x_norm) - 1, loc=m, scale=st.sem(x_norm))
            n_nan = len(x_nan)
            
            return {
                'n': n,
                'sum': tot,
                'mean': m,
                'SD': s,
                'med': me,
                'range': (l, u),
                'conf.': (conf_l, conf_u),
                'nan_count': n_nan
            }

In [3]:
p = os.path.join(PATH_INTERMEDIATE, 'stress-fixed.pkl')
X, y, groups, t, datetimes = load(p)

In [4]:
PARTICIPANTS = pd.read_csv(os.path.join(PATH_INTERMEDIATE, 'PARTICIPANT_INFO.csv'),index_col = 'pcode')
PINFO = PARTICIPANTS.assign(
    BFI_OPN=lambda x: x['openness'],
    BFI_CON=lambda x: x['conscientiousness'],
    BFI_NEU=lambda x: x['neuroticism'],
    BFI_EXT=lambda x: x['extraversion'],
    BFI_AGR=lambda x: x['agreeableness'],
)[[
    'BFI_OPN', 'BFI_CON', 'BFI_NEU', 'BFI_EXT', 'BFI_AGR'
]]
PINFO = pd.get_dummies(PINFO, prefix_sep='=', dtype=bool)

In [5]:
LABELS_PROC = pd.read_csv(os.path.join(PATH_INTERMEDIATE, 'LABELS_PROC.csv'), index_col=['pcode','timestamp'],parse_dates=True)

In [6]:
_df =LABELS_PROC
_df.reset_index(level='timestamp', inplace=True)
print('First timestamp:', _df['timestamp'].min())
print('Last timestamp:', _df['timestamp'].max())

First timestamp: 2019-04-30 10:03:28+09:00
Last timestamp: 2019-05-22 22:02:03+09:00


In [7]:
time_ranges = _df.groupby('pcode')['timestamp'].agg(['min', 'max'])

In [8]:
list_pid = set(LABELS_PROC.index.get_level_values('pcode').values)

In [9]:
PINFO_valid = PINFO.loc[PINFO.index.isin(list_pid)]

In [10]:
#Divide the features into different categories
feat_current = X.loc[:,[('#VAL' in str(x)) or ('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_dsc = X.loc[:,[('#DSC' in str(x))  for x in X.keys()]]  
feat_yesterday = X.loc[:,[('Yesterday' in str(x))  for x in X.keys()]]  
feat_today = X.loc[:,[('Today' in str(x))  for x in X.keys()]]  
feat_sleep = X.loc[:,[('Sleep' in str(x))  for x in X.keys()]]  
feat_time = X.loc[:,[('Time' in str(x))  for x in X.keys()]]  
feat_pif = X.loc[:,[('PIF' in str(x))  for x in X.keys()]]  
feat_ImmediatePast = X.loc[:,[('ImmediatePast_15' in str(x))  for x in X.keys()]]
#Divide the time window features into sensor/past stress label
feat_current_sensor = X.loc[:,[('#VAL' in str(x))  for x in X.keys()]]  
feat_current_ESM = X.loc[:,[('ESM#LastLabel' in str(x)) for x in X.keys()]]  
feat_ImmediatePast_sensor = feat_ImmediatePast.loc[:,[('ESM' not in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_ImmediatePast_ESM = feat_ImmediatePast.loc[:,[('ESM'  in str(x)) for x in feat_ImmediatePast.keys()]]  
feat_today_sensor = feat_today.loc[:,[('ESM' not in str(x))  for x in feat_today.keys()]]  
feat_today_ESM = feat_today.loc[:,[('ESM'  in str(x)) for x in feat_today.keys()]]  
feat_yesterday_sensor = feat_yesterday.loc[:,[('ESM' not in str(x)) for x in feat_yesterday.keys()]]  
feat_yesterday_ESM = feat_yesterday.loc[:,[('ESM'  in str(x)) for x in feat_yesterday.keys()]]
feat_baseline = pd.concat([ feat_time,feat_dsc,feat_current_sensor, feat_ImmediatePast_sensor],axis=1)
feat_final = pd.concat([feat_baseline  ], axis=1)
X = feat_final
cats = X.columns[X.dtypes == bool]

In [11]:
similar_user = pd.read_csv(os.path.join(PATH_INTERMEDIATE,  'similar_user_3.csv'))

In [12]:
import pandas as pd

# Convert groups to a pandas DataFrame
groups_df = pd.DataFrame(groups, columns=['pcode'])

# Ensure the data types of the 'pcode' and 'pcode' columns are the same
groups_df['pcode'] = groups_df['pcode'].astype(str)
similar_user['pcode'] = similar_user['pcode'].astype(str)

# Create a new Series that maps Pcode to cluster label
cluster_map = similar_user.set_index('pcode')['cluster']

# Create the 'cluster' column in the 'groups' DataFrame
groups_df['cluster'] = groups_df['pcode'].map(cluster_map)

In [13]:
cluster_map.head()

pcode
P01    2
P02    0
P03    2
P05    0
P06    1
Name: cluster, dtype: int64

In [14]:
# Create a dictionary to store data structures for each cluster
clusters = {}

# Loop through each unique cluster label in the Series
for cluster_label in cluster_map.unique():
    clusters[cluster_label] = cluster_map[cluster_map == cluster_label]

In [15]:
# XGBoost parameters
param = {
    "learning_rate": 0.01,
    "seed": RANDOM_STATE,
    "objective": 'binary:logistic',
    'eval_metric': 'auc',
    'verbosity': 0,
}

In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTENC, SMOTE
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold, TimeSeriesSplit, LeavePGroupsOut, train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

In [17]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone

normalize =True
# Feature selection method
selector = SelectFromModel(
        estimator=LogisticRegression(penalty='l1', solver='liblinear', C=1, random_state=RANDOM_STATE, max_iter=4000),
        threshold=0.005
#         estimator=LinearSVC(
#         penalty='l1',
#         loss='squared_hinge',
#         dual=False,
#         tol=1e-3,
#         C=1e-2,
#         max_iter=5000,
#         random_state=RANDOM_STATE
#     )  
    )
select = [clone(selector)]
oversample = True

In [18]:
C_cat = np.asarray(sorted(cats))
C_num = np.asarray(sorted(X.columns[~X.columns.isin(C_cat)]))

In [19]:
def process_dataframe(X):
    """
    Process the input DataFrame 'X':
    1. Set column names as a range from 0 to the number of columns.
    2. Replace boolean values with 1 for True and 0 for False.

    Parameters:
        X (pandas.DataFrame): The input DataFrame.

    Returns:
        pandas.DataFrame: The processed DataFrame.
    """
    # Set column names as a range from 0 to the number of columns
    X.columns = range(X.shape[1])

In [20]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score

# Store the results
results = {}

# Iterate through each cluster
for cluster_label, cluster_data in clusters.items():
    results[cluster_label] = {'accuracy': [], 'auc': [], 'f1': [], 'recall': [], 'F1 Positive': [], 'Precision': []}
    
    # Users in current cluster
    users_in_cluster = cluster_data.index.tolist()
    
    # Create a mask for data instances belonging to this cluster
    cluster_data_mask = groups_df['pcode'].isin(users_in_cluster)
    
    # Iterate through each user in the cluster for LOUO cross-validation
    for user in users_in_cluster:
        
        # Find the indices in groups_df for this user
        user_indices = groups_df[groups_df['pcode'] == user].index.tolist()
        
        # Mask for excluding the current user's data
        train_mask = (cluster_data_mask) & (~groups_df.index.isin(user_indices))
        
        # Split the data for LOUO cross-validation
        X_train = X[train_mask]
        y_train = y[train_mask.values]
        
        X_test = X.loc[user_indices]
        y_test = y[user_indices]
        
        X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_STATE,  stratify=y_train)
        
        if normalize:
            # Normalize numeric features
            scaler = StandardScaler().fit(X_train[C_num])
            X_train[C_num] = scaler.transform(X_train[C_num].copy())
            X_eval[C_num] = scaler.transform(X_eval[C_num].copy())
            X_test[C_num] = scaler.transform(X_test[C_num].copy())

        if select:
            if isinstance(select, SelectFromModel):
                select = [select]
            for i, s in enumerate(select):
                # Fit feature selector only on training data
                s.fit(X_train, y_train)
                selected_features = X_train.columns[s.get_support()].tolist()

                # Apply feature selection to train, eval, and test
                X_train = X_train[selected_features].copy()
                X_eval = X_eval[selected_features].copy()
                X_test = X_test[selected_features].copy()

        if oversample:
            # Determine categorical features for SMOTENC
            if len(C_cat):
                M = np.isin(X_train.columns, C_cat)
                sampler = SMOTENC(categorical_features=M, random_state=RANDOM_STATE)
            else:
                sampler = SMOTE(random_state = RANDOM_STATE)
            # Only oversample training data
            X_train, y_train = sampler.fit_resample(X_train, y_train)
        
        # Train the XGBoost model
        dtrain = xgb.DMatrix(X_train, label=y_train)
        deval = xgb.DMatrix(X_eval, label=y_eval)
        dtest = xgb.DMatrix(X_test, label=y_test)
        
        evallist = [(dtrain, 'train'), (deval, 'eval')]
        bst = xgb.train(param, dtrain, early_stopping_rounds=10, evals=evallist)
        
        y_real = dtest.get_label()  # Use the labels directly
        y_score = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
        
        y_pred = [1 if score >= 0.5 else 0 for score in y_score]
        
        # Calculate the metrics
        accuracy = accuracy_score(y_real, y_pred)
        auc = roc_auc_score(y_true=y_real, y_score=y_score, average=None)
        f1 = f1_score(y_true=y_real, y_pred=y_pred, pos_label=1, average='macro', zero_division=0)
        recall = recall_score(y_true=y_real, y_pred=y_pred, pos_label=1, average='macro', zero_division=0)
        f1p = f1_score(y_true=y_real, y_pred=y_pred, pos_label=1, average='binary', zero_division=0)
        precision = precision_score(y_true=y_real, y_pred=y_pred, pos_label=1, average='macro', zero_division=0)
        
        # Store the metrics
        results[cluster_label]['accuracy'].append(accuracy)
        results[cluster_label]['auc'].append(auc)
        results[cluster_label]['f1'].append(f1)
        results[cluster_label]['recall'].append(recall)
        results[cluster_label]['F1 Positive'].append(f1p)
        results[cluster_label]['Precision'].append(precision)

# results dictionary will have the metrics for each user in each cluster

[0]	train-auc:0.92807	eval-auc:0.60777
[1]	train-auc:0.93969	eval-auc:0.61953
[2]	train-auc:0.94410	eval-auc:0.61953
[3]	train-auc:0.94504	eval-auc:0.61305
[4]	train-auc:0.94429	eval-auc:0.60566
[5]	train-auc:0.94554	eval-auc:0.61463
[6]	train-auc:0.94634	eval-auc:0.61780
[7]	train-auc:0.94733	eval-auc:0.62149
[8]	train-auc:0.94776	eval-auc:0.62044
[9]	train-auc:0.94754	eval-auc:0.62059


/home/uzair/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[0]	train-auc:0.89803	eval-auc:0.64808
[1]	train-auc:0.89888	eval-auc:0.64631
[2]	train-auc:0.90081	eval-auc:0.64123
[3]	train-auc:0.90026	eval-auc:0.64385
[4]	train-auc:0.90217	eval-auc:0.64192
[5]	train-auc:0.90210	eval-auc:0.64192
[6]	train-auc:0.90292	eval-auc:0.63992
[7]	train-auc:0.90330	eval-auc:0.63954
[8]	train-auc:0.93668	eval-auc:0.64931
[9]	train-auc:0.93851	eval-auc:0.64823


/home/uzair/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[0]	train-auc:0.91299	eval-auc:0.59994
[1]	train-auc:0.92058	eval-auc:0.61575
[2]	train-auc:0.91900	eval-auc:0.62076
[3]	train-auc:0.91908	eval-auc:0.62244
[4]	train-auc:0.92329	eval-auc:0.62145
[5]	train-auc:0.92361	eval-auc:0.62114
[6]	train-auc:0.92883	eval-auc:0.62874
[7]	train-auc:0.93306	eval-auc:0.62973
[8]	train-auc:0.93522	eval-auc:0.62958
[9]	train-auc:0.93570	eval-auc:0.62623


...

/home/uzair/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[0]	train-auc:0.92199	eval-auc:0.64739
[1]	train-auc:0.92504	eval-auc:0.63202
[2]	train-auc:0.92660	eval-auc:0.61469
[3]	train-auc:0.92885	eval-auc:0.61594
[4]	train-auc:0.92541	eval-auc:0.61247
[5]	train-auc:0.92562	eval-auc:0.62020
[6]	train-auc:0.92522	eval-auc:0.61469
[7]	train-auc:0.92589	eval-auc:0.62056
[8]	train-auc:0.92666	eval-auc:0.61549
[9]	train-auc:0.92669	eval-auc:0.61532


/home/uzair/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


[0]	train-auc:0.92999	eval-auc:0.63649
[1]	train-auc:0.93685	eval-auc:0.62397
[2]	train-auc:0.93794	eval-auc:0.62853
[3]	train-auc:0.94821	eval-auc:0.66038
[4]	train-auc:0.94480	eval-auc:0.65501
[5]	train-auc:0.95334	eval-auc:0.65832
[6]	train-auc:0.95348	eval-auc:0.66225
[7]	train-auc:0.95577	eval-auc:0.65653
[8]	train-auc:0.95333	eval-auc:0.66404
[9]	train-auc:0.95557	eval-auc:0.65921


/home/uzair/miniconda3/envs/sci-data/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [21]:
# Store average results for each cluster
average_results_per_cluster = {}

for cluster_label, metrics in results.items():
    average_results_per_cluster[cluster_label] = {
        'avg_accuracy': sum(metrics['accuracy']) / len(metrics['accuracy']),
        'avg_auc': sum(metrics['auc']) / len(metrics['auc']),
        'avg_f1': sum(metrics['f1']) / len(metrics['f1']),
        'avg_recall': sum(metrics['recall']) / len(metrics['recall']),
        'avg_fp1':sum(metrics['F1 Positive']) / len(metrics['F1 Positive']),
        'avg_pre':sum(metrics['Precision']) / len(metrics['Precision'])
    }

# Calculate overall averages
all_accuracies = [avg['avg_accuracy'] for cluster, avg in average_results_per_cluster.items()]
all_aucs = [avg['avg_auc'] for cluster, avg in average_results_per_cluster.items()]
all_f1s = [avg['avg_f1'] for cluster, avg in average_results_per_cluster.items()]
all_recalls = [avg['avg_recall'] for cluster, avg in average_results_per_cluster.items()]
all_fp1 = [avg['avg_fp1'] for cluster, avg in average_results_per_cluster.items()]
all_pre = [avg['avg_pre'] for cluster, avg in average_results_per_cluster.items()]

print("\n")

overall_avg_results = {
    'overall_avg_accuracy': sum(all_accuracies) / len(all_accuracies),
    'overall_avg_auc': sum(all_aucs) / len(all_aucs),
    'overall_avg_f1': sum(all_f1s) / len(all_f1s),
    'overall_avg_recall': sum(all_recalls) / len(all_recalls),
    'overall_avg_f1p' : sum(all_fp1) / len(all_fp1),
    'overall_avg_pre' : sum(all_pre) / len(all_pre)
}

print(average_results_per_cluster)
print("\n")
print(overall_avg_results)



{2: {'avg_accuracy': 0.5376149983176912, 'avg_auc': 0.4920580409184682, 'avg_f1': 0.4449902139968375, 'avg_recall': 0.47407937083956386, 'avg_fp1': 0.28406282380072856, 'avg_pre': 0.4832865550219083}, 0: {'avg_accuracy': 0.5133018708256417, 'avg_auc': 0.5225880985292437, 'avg_f1': 0.4702400151212414, 'avg_recall': 0.5052543796483762, 'avg_fp1': 0.3514465558802181, 'avg_pre': 0.5043785445651633}, 1: {'avg_accuracy': 0.5116192355054808, 'avg_auc': 0.5292789876602001, 'avg_f1': 0.4572959452140577, 'avg_recall': 0.5211456474120524, 'avg_fp1': 0.38153066769925287, 'avg_pre': 0.5189874723783882}}


{'overall_avg_accuracy': 0.5208453682162713, 'overall_avg_auc': 0.5146417090359706, 'overall_avg_f1': 0.45750872477737886, 'overall_avg_recall': 0.5001597992999974, 'overall_avg_f1p': 0.33901334912673314, 'overall_avg_pre': 0.5022175239884866}
